In [1]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
import tensorflow as tf

In [2]:
model = load_model(r'C:\Users\TANAY\Face_mask_Detector_model.h5')

In [3]:
classifier = cv2.CascadeClassifier('D:\Face Mask detection\haarcascade_frontalface_default.xml')

In [4]:
size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

In [5]:
text_dict = {0:'Mask off' , 1:'Mask 0n'}
rect_color_dict = {0:(0,0,255),1:(0,255,0)}

In [6]:
while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        img= im[y:y+h, x:x+w]
        img=cv2.resize(img,(150,150))
        img=img/255.0
        img=np.reshape(img,(1,150,150,3))
        img = np.array(img)
        result=model.predict(img)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),rect_color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),rect_color_dict[label],-1)
        cv2.putText(im, text_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()